#### Import libraries

In [1]:
import pandas as pd

#### Load data

In [12]:
df = pd.read_csv('./data/data.csv')

#### Clean data

In [20]:
df['land_size_sqm'] = df['land_size_sqm'].str.replace(' sqm', '')
df['house_size_sqm'] = df['house_size_sqm'].str.replace(' sqm', '')
df['large_living_room'] = df['large_living_room'].str.replace('No', '0')
df['large_living_room'] = df['large_living_room'].str.replace('Yes', '1')
df['parking_space'] = df['parking_space'].str.replace('No', '0')
df['parking_space'] = df['parking_space'].str.replace('Yes', '1')

In [24]:
df.head()

,land_size_sqm,house_size_sqm,no_of_rooms,no_of_bathrooms,large_living_room,parking_space,front_garden,swimming_pool,distance_to_school,wall_fence,house_age,water_front,distance_to_supermarket,crime_rate_index,room_size,property_value
0,201,177,3,1,0,1,Yes,No,3.3km,Yes,10 Years,No,6.8 km,0.90,small,165432
1,196,182,4,3,1,1,No,Yes,1.2km,Yes,11 Years,No,4.1 km,1.42,medium,187043
2,198,182,4,4,1,1,No,Yes,5.9km,No,20 Years,No,2.1 km,4.12,medium,148658
3,178,166,2,3,0,1,No,No,5.9km,No,5 Years,No,0.7 km,4.36,small,123785
4,183,165,3,1,1,1,No,No,3.8km,Yes,8 Years,No,0.7 km,0.42,small,156470
